In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import importlib

from Demos.mmapfile_demo import move_src
from dipy.utils.parallel import joblib

import combined_analysis_bachelor.code.functions_for_pipeline as funcs
import combined_analysis_bachelor.code.movement_detection_functions as move_functions
import combined_analysis_bachelor.code.cnt_to_other_format as cnt_to_raw
import combined_analysis_bachelor.code.create_processed_files as process
import my_utils.find_paths as find_path
from my_utils.get_sub_dir import get_sub_folder_dir

C:\Users\User\AppData\Local\Temp\tmpzrabb_jy 32768 tmpzrabb_jy


In [3]:
fig_path = find_path.get_onedrive_path(folder="figures")

project folder found: C:\Users\User\OneDrive - Charité - Universitätsmedizin Berlin\TRR295 via Teams - LID_MEG - LID_MEG


### 1. step: trim recording and run through processing pipeline ###

In [5]:
source_directory = "E:\external_val_cnt"
class_dir = get_sub_folder_dir("classification", "classification_data")
target_dir = f"{class_dir}/class_raw/external_val_data_raw2"

SUB = "class"
LOCS = {"BIP7": "brachioradialis_R",
        "BIP8": "deltoideus_R",
        "BIP3": "acc_x_hand_R",
        "BIP4": "acc_y_hand_R",
        "BIP5": "acc_z_hand_R",
        }
channels_c = ["BIP7", "BIP8","BIP3", "BIP4", "BIP5"]
EMG = ["BIP7", "BIP8"]
SF = 1000

project folder found: C:\Users\User\OneDrive - Charité - Universitätsmedizin Berlin\TRR295 via Teams - LID_MEG - LID_MEG
C:\Users\User\OneDrive - Charité - Universitätsmedizin Berlin\TRR295 via Teams - LID_MEG - LID_MEG\data\classification_data


In [2]:
#cnt_to_raw.cnt_to_raw_hdf(SUB, source_directory, target_dir, channels_c, EMG, LOCS, SF)

In [6]:
source_dir1 = f"{class_dir}/class_raw/external_val_data_raw"
target_dir1 = f"{class_dir}/class_processed/external_val_data_proc"

source_dir2 = f"{class_dir}/class_raw/external_val_data_raw2"
target_dir2 = f"{class_dir}/class_processed/external_val_data_proc2"

In [136]:
importlib.reload(process)

<module 'combined_analysis_bachelor.code.create_processed_files' from 'C:\\Users\\User\\Documents\\GitHub\\lid_opm\\combined_analysis_bachelor\\code\\create_processed_files.py'>

In [1]:
#process.create_processed_files(source_dir2, target_dir2, [2,20], [20,450], SF)

In [8]:
external_test8 = pd.read_hdf(f"{target_dir2}/sub-class_test8_processed.h5", key="data")

In [6]:
external_test8

,brachioradialis_R,deltoideus_R,acc_x_hand_R,acc_y_hand_R,acc_z_hand_R,Sync_Time (s),SVM
0,4.742873e-13,-6.061818e-14,-2.949030e-17,1.864828e-17,2.450297e-17,0.000,4.263604e-17
1,2.460697e+00,6.365698e-01,1.367903e-03,-3.835716e-05,-7.654852e-04,0.001,1.567991e-03
2,8.426906e+00,9.258400e-01,2.713657e-03,-7.844021e-05,-1.527131e-03,0.002,3.114838e-03
3,-1.340550e+00,3.199888e-02,4.015248e-03,-1.217082e-04,-2.280910e-03,0.003,4.619478e-03
4,1.432336e+01,-2.322987e+01,5.250851e-03,-1.698382e-04,-3.023103e-03,0.004,6.061306e-03
...,...,...,...,...,...,...,...
35777,1.556980e+01,-9.350170e+00,1.442178e-03,7.451517e-04,-2.518661e-03,35.777,2.996461e-03
35778,3.822128e+00,-3.760055e+00,1.118324e-03,5.674784e-04,-1.908244e-03,35.778,2.283435e-03
35779,-1.088326e+01,1.130868e+01,7.631618e-04,3.824863e-04,-1.281449e-03,35.779,1.539748e-03
35780,-9.858548e+00,1.593713e+01,3.868909e-04,1.924377e-04,-6.434994e-04,35.780,7.751182e-04


In [9]:
len(external_test8["Sync_Time (s)"])

35782

In [10]:
df_test_proc8_crop_end = funcs.crop_edges(external_test8, crop_only_end=True)

In [11]:
len(df_test_proc8_crop_end["Sync_Time (s)"])

34782

In [12]:
dfs = [df_test_proc8_crop_end]

In [13]:
#for df in dfs:
 #   df["SVM"] = df["SVM_R"]
 #   df.drop("SVM_R", axis=1, inplace=True)
EMG_cols = ["brachioradialis_R", "deltoideus_R"]
ACC_col = ["SVM", "acc_x_hand_R", "acc_y_hand_R", "acc_z_hand_R"] 

In [14]:
dfs_proc = []
for df in dfs:
    df_with_tkeo = funcs.add_tkeo_add_envelope(df, EMG_cols, 20, 3, 1000)
    df_processed = df_with_tkeo
    print(len(df_processed["Sync_Time (s)"]))
    df_proc = df_processed.dropna(how="any", ignore_index=True)
    print(len(df_proc["Sync_Time (s)"]))
    
    fig, axs = plt.subplots(4, 1)
    axs = axs.ravel()
    axs[0].plot(df_proc["Sync_Time (s)"], df_proc["brachioradialis_R_tkeo_rms"], label="brachioradialis tkeo")
    axs[0].set_xlabel("time (s)")
    axs[0].legend()
    axs[1].plot(df_proc["Sync_Time (s)"], df_proc["deltoideus_R_tkeo_rms"], label="delt tkeo")
    axs[1].set_xlabel("time (s)")
    axs[1].legend()
    #axs[2].plot(df_proc["Sync_Time (s)"], df_proc["SVM_smooth_rms"], label="SVM")
    axs[2].plot(df_proc["Sync_Time (s)"], df_proc["SVM"], label="SVM")
    axs[2].set_xlabel("time (s)")
    axs[2].legend()
    #axs[3].plot(df_proc["Sync_Time (s)"], df_proc["acc_z_hand_R_smooth_rms"], label="acc_z_R")
    axs[3].plot(df_proc["Sync_Time (s)"], df_proc["acc_z_hand_R"], label="acc_z_R")
    axs[3].set_xlabel("time (s)")
    axs[3].legend()
    plt.show()
    
    dfs_proc.append(df_proc)

34782
34782


In [210]:
importlib.reload(funcs)
importlib.reload(move_functions)

<module 'combined_analysis_bachelor.code.movement_detection_functions' from 'C:\\Users\\User\\Documents\\GitHub\\lid_opm\\combined_analysis_bachelor\\code\\movement_detection_functions.py'>

In [15]:
baselines = {
             "df_test8": [1,7],
             }

df_test_proc8 = dfs_proc[0]
print(len(df_test_proc8["Sync_Time (s)"]))

channels = ["brachioradialis_R_tkeo_rms", "deltoideus_R_tkeo_rms", "SVM"]

34782


In [90]:
plt.figure()
plt.plot(external_test8["Sync_Time (s)"], external_test8["deltoideus_R"])
plt.plot(df_test_proc8["Sync_Time (s)"], df_test_proc8["deltoideus_R_tkeo_rms"])
plt.show()

In [31]:
dict_onsets_offsets_test8_threshed = funcs.simple_interactive_multichannel_plot(df_test_proc8, channels, baselines, "df_test8", 1000, give_out_dict=True)

KeyboardInterrupt: 

In [20]:
dict_onsets_offsets_test8_threshed = {'brachioradialis_R_tkeo_rms': [[9194, 9304], [9377, 11632], [17698, 20506]],
 'deltoideus_R_tkeo_rms': [[24278, 25693], [28701, 30477]],
 'SVM': [[9431, 9770],
  [11744, 11879],
  [23815, 23944],
  [24019, 24558],
  [24633, 24795],
  [24875, 26245],
  [26320, 26472],
  [26535, 26706],
  [28345, 30685],
  [30812, 31118]]}

# values that were used were:
# SVM = 6, 0.1, 0.05
# delt = 15, 0.1, 0.05
# brachio = 10, 0.1, 0.05

In [21]:
# behavioral labels für thresh raus kriegen:
delt_activity_test8_thresh = move_functions.fill_activity_mask(dict_onsets_offsets_test8_threshed["deltoideus_R_tkeo_rms"], 1000, df_test_proc8["Sync_Time (s)"])
brachioradialis_activity_test8_thresh = move_functions.fill_activity_mask(dict_onsets_offsets_test8_threshed["brachioradialis_R_tkeo_rms"], 1000, df_test_proc8["Sync_Time (s)"])
acc_activity_test8_thresh = move_functions.fill_activity_mask(dict_onsets_offsets_test8_threshed["SVM"], 1000, df_test_proc8["Sync_Time (s)"])

behavioral_labels_thresh = move_functions.create_behavioral_array_arm(delt_activities_binary=delt_activity_test8_thresh, brachioradialis_activities_binary=brachioradialis_activity_test8_thresh, acc_activities_binary=acc_activity_test8_thresh)

In [19]:
# das vllt einfach mit reinnehmen als 2. subplot zu y_pred/test kombi??

plt.figure()
plt.plot(behavioral_labels_thresh)
plt.show()

In [22]:
dict_onsets_offsets_test8_ground_truth = {'brachioradialis_R_tkeo_rms': [[9010, 11850], [17350, 20900]],
 'deltoideus_R_tkeo_rms': [[24200, 25700], [28700, 30500]],
 'SVM': [[23500, 27300], [28230, 31500]]}

In [23]:
delt_activity_test8_ground_truth = move_functions.fill_activity_mask(dict_onsets_offsets_test8_ground_truth["deltoideus_R_tkeo_rms"], 1000, df_test_proc8["Sync_Time (s)"])
brachioradialis_activity_test8_ground_truth = move_functions.fill_activity_mask(dict_onsets_offsets_test8_ground_truth["brachioradialis_R_tkeo_rms"], 1000, df_test_proc8["Sync_Time (s)"])
acc_activity_test8_ground_truth = move_functions.fill_activity_mask(dict_onsets_offsets_test8_ground_truth["SVM"], 1000, df_test_proc8["Sync_Time (s)"])

In [24]:
behavioral_labels_ground_truth = move_functions.create_behavioral_array_arm(delt_activities_binary=delt_activity_test8_ground_truth, brachioradialis_activities_binary=brachioradialis_activity_test8_ground_truth, acc_activities_binary=acc_activity_test8_ground_truth)

In [203]:
import matplotlib as mpl
import matplotlib.pyplot as plt

plt.style.use("default")
mpl.rcParams.update(mpl.rcParamsDefault)

# Spines/Ticks explizit global einschalten (für neuere MPL-Versionen)
mpl.rcParams.update({
    "axes.linewidth": 1.2,
    "axes.edgecolor": "black",
    "axes.spines.left":   False,
    "axes.spines.right":  False,
    "axes.spines.top":    False,
    "axes.spines.bottom": True,
})

In [208]:
thresh_GT_fig_path = f"{fig_path}/mix_EMG_ACC_figures/compare_thresh_GT.svg"

order = ['rest', 'move', 'suppr']
enc = {lbl: i for i, lbl in enumerate(order)}

fig, ax = plt.subplots(1,1, figsize=(12,6), constrained_layout=True)

ax.set_facecolor("white")

plt.step(df_test_proc8["Sync_Time (s)"], behavioral_labels_ground_truth, color="lightgray", linewidth=5, label="Ground truth")
plt.step(df_test_proc8["Sync_Time (s)"], behavioral_labels_thresh, color="purple", alpha=0.9, linewidth=1.1, label="TBM labels")

ax.set_xlabel("Time (s)")
ax.set_yticklabels(order) 
ax.set_ylim(-0.25, 2.25)
ax.set_yticks([0,1,2])
sp = ax.spines["bottom"]
sp.set_visible(True)
sp.set_linewidth(0.8)
sp.set_color("black")
sp.set_zorder(10)  
ax.tick_params(bottom=True, left=True, )  

#plt.suptitle("Comparison of Ground Truth labels and the ones generated with the ST method")

ax.legend(bbox_to_anchor=(1.02,0.5), loc="center left", frameon=False)
plt.tight_layout()
#plt.show()
plt.savefig(thresh_GT_fig_path, format="svg", dpi=300, bbox_inches="tight", pad_inches=0.05)

C:\Users\User\AppData\Local\Temp\ipykernel_8556\414298283.py:14: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_yticklabels(order)
C:\Users\User\AppData\Local\Temp\ipykernel_8556\414298283.py:27: UserWarning: The figure layout has changed to tight
  plt.tight_layout()


In [67]:
thresh_correct = behavioral_labels_thresh == behavioral_labels_ground_truth

PRED_ACC_thresh = sum(thresh_correct) / len(thresh_correct)

print(PRED_ACC_thresh)

print(round(PRED_ACC_thresh, 3))

0.8999482490943591
0.9


In [69]:
print(len(behavioral_labels_ground_truth))

34782


In [23]:
plt.plot(behavioral_labels_ground_truth)

plt.show()

In [314]:
print(len(df_test_proc8_crop_end["Sync_Time (s)"]))

34782


In [311]:
import my_utils.find_paths as find_path

fig_path = find_path.get_onedrive_path(folder="figures")
saving_path = f"{fig_path}/mix_EMG_ACC_figures/external_val_onoffsets_threshed.svg"

ground_truth_path = f"{fig_path}/mix_EMG_ACC_figures/external_val_onoffsets_ground_truth.svg"

project folder found: C:\Users\User\OneDrive - Charité - Universitätsmedizin Berlin\TRR295 via Teams - LID_MEG - LID_MEG


In [321]:
importlib.reload(funcs)

<module 'combined_analysis_bachelor.code.functions_for_pipeline' from 'C:\\Users\\User\\Documents\\GitHub\\lid_opm\\combined_analysis_bachelor\\code\\functions_for_pipeline.py'>

In [322]:
# ground truth

funcs.plot_with_behavior(df_test_proc8_crop_end, behavioral_labels_ground_truth, dict_onsets_offsets_test8_ground_truth["deltoideus_R_tkeo_rms"], dict_onsets_offsets_test8_ground_truth["brachioradialis_R_tkeo_rms"], dict_onsets_offsets_test8_ground_truth["SVM"], save=False, save_path=ground_truth_path)

In [310]:
# threshed

funcs.plot_with_behavior(df_test_proc8_crop_end, behavioral_labels_thresh, dict_onsets_offsets_test8_threshed["deltoideus_R_tkeo_rms"], dict_onsets_offsets_test8_threshed["brachioradialis_R_tkeo_rms"], dict_onsets_offsets_test8_threshed["SVM"], save=True, save_path=saving_path)

In [71]:
df_test_proc8.keys()

Index(['brachioradialis_R', 'deltoideus_R', 'acc_x_hand_R', 'acc_y_hand_R',
       'acc_z_hand_R', 'Sync_Time (s)', 'SVM', 'brachioradialis_R_tkeo_rms',
       'brachioradialis_R_envelope', 'deltoideus_R_tkeo_rms',
       'deltoideus_R_envelope'],
      dtype='object')

In [17]:
df_test_proc8_final = df_test_proc8.drop(["acc_x_hand_R", "acc_y_hand_R", "acc_z_hand_R", 'Sync_Time (s)'], axis=1)

In [18]:
df_test_proc8_final.keys()

Index(['brachioradialis_R', 'deltoideus_R', 'SVM',
       'brachioradialis_R_tkeo_rms', 'brachioradialis_R_envelope',
       'deltoideus_R_tkeo_rms', 'deltoideus_R_envelope'],
      dtype='object')

In [ ]:
# exporting external df for processing and testing

In [16]:
from sklearn.preprocessing import StandardScaler
import os
import joblib

In [25]:
importlib.reload(move_functions)


# create windowing, calculate features from windows
X_test, y_test_true_nr, y_coding =  move_functions.create_test_df(df=df_test_proc8_final,
                                         output_dir=False,
                                         behavioral_array=behavioral_labels_ground_truth)

# convert labels to string labels
y_test_true = [y_coding[y] for y in y_test_true_nr]


final_feats_new = ['deltoideus_envelope_mav', 'brachioradialis_tkeo_rms_std', 'brachioradialis_tkeo_rms_mav', 'SVM_range', 'brachioradialis_waveformLength', 'SVM_mean', 'brachioradialis_envelope_std', 'brachioradialis_envelope_rms', 'brachioradialis_rms', 'deltoideus_waveformLength', 'brachioradialis_max', 'brachioradialis_tkeo_rms_max', 'brachioradialis_envelope_coefVar', 'SVM_psd_peakfreq', 'deltoideus_envelope_rms', 'brachioradialis_std', 'brachioradialis_envelope_max', 'deltoideus_slopeSignChanges', 'SVM_rms', 'SVM_max', 'brachioradialis_envelope_mav', 'deltoideus_mav', 'SVM_min', 'brachioradialis_mav', 'SVM_std', 'brachioradialis_slopeSignChanges', 'brachioradialis_tkeo_rms_rms']

drop_feats = [i for i in X_test.columns if i not in final_feats_new + ["label", "sub_ID", ]]
print(len(drop_feats))
print(len(final_feats_new))
X_test = X_test.drop(drop_feats, axis=1)
print(X_test.shape)

# standardise features 
scaler = StandardScaler()
X_test = scaler.fit_transform(X_test)


17
27
(277, 27)


In [26]:
X_test

array([[ 0.55705466,  0.74323316,  0.55657788, ..., -0.52552189,
        -0.28683048, -0.89780902],
       [ 0.22099774,  0.33923711,  0.22123926, ..., -0.35429135,
        -0.54228643, -0.89780902],
       [-0.16575522, -0.12039153, -0.16549181, ..., -0.48818274,
        -0.41350933,  1.68825439],
       ...,
       [-0.47923642, -0.47224747, -0.47908544, ..., -0.40246017,
        -0.26995138, -0.89780902],
       [-0.41351563, -0.41867779, -0.41324935, ..., -0.36458679,
        -0.28060117, -0.03578788],
       [-0.45397756, -0.44933091, -0.4537108 , ..., -0.26657806,
        -0.39780784, -0.89780902]])

In [26]:
X_test.shape

(277, 27)

In [77]:
y_coding

{-1: 'unknown', 0: 'rest', 1: 'move', 2: 'suppr'}

In [78]:
plt.plot(y_test_true_nr)

plt.show()

Load Model

In [27]:
model_dir = get_sub_folder_dir("classification", "classification_data")
model_dir = os.path.join(model_dir, "models")

model_names = {"logreg": "final_logreg_new.pkl", "knn": "final_knn_new.pkl", "rf": "final_rf_new.pkl"}

project folder found: C:\Users\User\OneDrive - Charité - Universitätsmedizin Berlin\TRR295 via Teams - LID_MEG - LID_MEG
C:\Users\User\OneDrive - Charité - Universitätsmedizin Berlin\TRR295 via Teams - LID_MEG - LID_MEG\data\classification_data


In [28]:
os.path.join(model_dir, model_names["logreg"])

'C:\\Users\\User\\OneDrive - Charité - Universitätsmedizin Berlin\\TRR295 via Teams - LID_MEG - LID_MEG\\data\\classification_data\\sub-classification\\EMG_ACC\\models\\final_logreg_new.pkl'

In [29]:
with open(os.path.join(model_dir, model_names["logreg"]), 'rb') as f:
    lr_cls = joblib.load(f)

In [30]:
y_pred_lr = lr_cls.predict(X_test)

y_pred_lr

array(['move', 'move', 'suppr', 'suppr', 'suppr', 'suppr', 'suppr',
       'suppr', 'suppr', 'suppr', 'suppr', 'suppr', 'suppr', 'suppr',
       'suppr', 'suppr', 'suppr', 'suppr', 'suppr', 'rest', 'rest',
       'suppr', 'suppr', 'rest', 'rest', 'suppr', 'suppr', 'suppr',
       'suppr', 'rest', 'rest', 'suppr', 'suppr', 'rest', 'suppr',
       'suppr', 'move', 'suppr', 'suppr', 'suppr', 'suppr', 'rest',
       'rest', 'suppr', 'rest', 'rest', 'rest', 'suppr', 'suppr', 'suppr',
       'rest', 'rest', 'suppr', 'rest', 'suppr', 'suppr', 'rest', 'rest',
       'rest', 'rest', 'rest', 'rest', 'rest', 'rest', 'rest', 'rest',
       'rest', 'rest', 'rest', 'rest', 'suppr', 'suppr', 'suppr', 'suppr',
       'suppr', 'suppr', 'suppr', 'suppr', 'suppr', 'suppr', 'suppr',
       'suppr', 'suppr', 'suppr', 'suppr', 'suppr', 'suppr', 'suppr',
       'suppr', 'suppr', 'suppr', 'suppr', 'suppr', 'move', 'suppr',
       'suppr', 'suppr', 'suppr', 'suppr', 'suppr', 'suppr', 'suppr',
       'suppr', '

In [83]:
plt.plot(y_pred_lr, color='blue')
plt.plot(y_test_true, color='green')

plt.show()

In [31]:
y_correct_lr = y_pred_lr == y_test_true

PRED_ACC_lr = sum(y_correct_lr) / len(y_correct_lr)

In [32]:
PRED_ACC_lr

0.5234657039711191

In [86]:
%matplotlib inline

In [177]:
def force_axes(ax, lw=0.8, color="black"):
    # falls irgendwo axis('off') oder frame_off gesetzt wurde
    ax.set_axis_on()
    ax.set_frame_on(True)
    # Spines erzwingen
    for side in ("bottom","left"):
        sp = ax.spines[side]
        sp.set_visible(True)
        sp.set_linewidth(lw)
        sp.set_color(color)
        sp.set_zorder(10)  

    ax.tick_params(bottom=True, left=True, ) 
    
def force_axes_onlyB(ax, lw=0.8, color="black"):
    # falls irgendwo axis('off') oder frame_off gesetzt wurde
    ax.set_axis_on()
    ax.set_frame_on(True)
    for side in ("bottom"):
        sp = ax.spines[side]
        sp.set_visible(True)
        sp.set_linewidth(lw)
        sp.set_color(color)
        sp.set_zorder(10)  

    ax.tick_params(bottom=True, left=True, )  

Significance testing

In [33]:
n_perm = 10000

def significance_test_random(n_permutations, y_true, y_pred, y_encoding):
    random_accuracies = []
    
    # np.random.seed(42)
    
    for i in np.arange(n_permutations):
        y_random = np.random.randint(0, high=3, size=len(y_pred))
        y_random = [y_encoding[y] for y in y_random]
    
        temp_acc = np.array(y_random) == np.array(y_true)
        temp_acc = sum(temp_acc) / len(y_random)
        random_accuracies.append(temp_acc)
        
    
    return random_accuracies

In [94]:
rand_accuracies_lr = significance_test_random(n_perm, y_true=y_test_true, y_pred=y_pred_lr, y_encoding=y_coding)

In [35]:
alpha = 0.01

sign_border_lr = np.percentile(rand_accuracies_lr, 99)

print(sign_border_lr)

0.4007220216606498


In [292]:
import matplotlib.lines as mlines
import matplotlib.patches as mpatches

lr_fig_path = f"{fig_path}/mix_EMG_ACC_figures/external_val_permutation_hist_LR.svg"


def plot_acc_hist(sign_border_model, PRED_ACC, rand_accuracies, path, model_name=""):
    fig, ax = plt.subplots(1,1, figsize=(6,4), constrained_layout=False)
    
    plt.hist(rand_accuracies, color="gray", alpha=0.8)
    
    x_right = max(rand_accuracies) * 1.25
    ax.set_xlim(right=x_right)
    ax.axvspan(sign_border_model, x_right, color="lightgray", alpha=0.4, zorder=0)
    
    plt.axvline(sign_border_model, color="black", label="significance border (a = 0.01)") 
    plt.axvline(PRED_ACC, color="red", alpha=0.8, label=f"Pred accuracy with  model {model_name} (p = 0.0001)")
    
    ax.set_facecolor("white")
    ax.set_ylabel("Frequency")
    ax.set_xlabel("Accuracy values")
    plt.xticks([0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55])
    plt.yticks([1000, 2000, 3000])
    
    h_sign = mlines.Line2D([], [], color='black', linewidth=3, label='significance border (0.01)')
    h_pred = mlines.Line2D([], [], color='red',   linewidth=3, label='Pred. accuracy (p = 0.0001)')
    h_region = mpatches.Patch(color="lightgray", alpha=0.4, label="highly significant region")
    
    #ax.legend(handles=[h_sign, h_pred, h_region],
     #          loc='upper left', bbox_to_anchor=(1.02, 0.5),
      #        frameon=False,   
      #        handlelength=2.5,
      #        handletextpad=0.6,
      #        borderpad=0.2)
    
    leg_ax = fig.add_axes([0.82, 0.15, 0.16, 0.70])  # [left, bottom, width, height]
    leg_ax.axis("off")
    leg_ax.legend(handles=[h_sign, h_pred, h_region],
                  loc="center left",
                  frameon=False)

    # Hauptplot etwas schmaler machen
    fig.subplots_adjust(right=0.80)
    
    ax.grid(False)
    force_axes(ax)
    #plt.show()
    #plt.tight_layout()
    plt.savefig(path) # , bbox_inches="tight", bbox_extra_artists=(ax.get_legend(),))

In [293]:
plot_acc_hist(sign_border_lr, PRED_ACC_lr, rand_accuracies_lr, lr_fig_path)

In [90]:
p = sum(rand_accuracies_lr > PRED_ACC_lr) + 1/ (n_perm + 1)
p = round(p, 4)

print(p)

0.0001


repeat with KNN

In [134]:
with open(os.path.join(model_dir, model_names["knn"]), 'rb') as f:
    knn_cls = joblib.load(f)

In [135]:
y_pred_knn = knn_cls.predict(X_test)

In [47]:
plt.plot(y_pred_knn, color='blue')
plt.plot(y_test_true, color='green')

plt.show()

In [136]:
y_correct_knn = y_pred_knn == y_test_true

PRED_ACC_knn = sum(y_correct_knn) / len(y_correct_knn)

In [137]:
print(PRED_ACC_knn)

0.5379061371841155


In [138]:
rand_accuracies_knn = significance_test_random(n_perm, y_true=y_test_true, y_pred=y_pred_knn, y_encoding=y_coding)

In [139]:
alpha = 0.01

sign_border_knn = np.percentile(rand_accuracies_knn, 99)

print(sign_border_knn)

0.4007220216606498


In [140]:
p = sum(rand_accuracies_knn > PRED_ACC_knn) + 1/ (n_perm + 1)
p = round(p, 4)

print(p)

0.0001


In [142]:
knn_fig_path = f"{fig_path}/mix_EMG_ACC_figures/external_val_permutation_hist_KNN.svg"

plot_acc_hist(sign_border_knn, PRED_ACC_knn, rand_accuracies_knn, knn_fig_path)

repeat with RF

In [143]:
with open(os.path.join(model_dir, model_names["rf"]), 'rb') as f:
    rf_cls = joblib.load(f)

In [144]:
y_pred_rf = rf_cls.predict(X_test)

In [145]:
plt.plot(y_pred_rf, color='blue')
plt.plot(y_test_true, color='green')

plt.show()

In [146]:
y_correct_rf = y_pred_rf == y_test_true

PRED_ACC_rf = sum(y_correct_rf) / len(y_correct_rf)

In [147]:
print(PRED_ACC_rf)

0.5306859205776173


In [148]:
rand_accuracies_rf = significance_test_random(n_perm, y_true=y_test_true, y_pred=y_pred_rf, y_encoding=y_coding)

In [149]:
alpha = 0.01

sign_border_rf = np.percentile(rand_accuracies_rf, 99)

print(sign_border_rf)

0.4007220216606498


In [150]:
p = sum(rand_accuracies_rf > PRED_ACC_rf) + 1/ (n_perm + 1)
p = round(p, 4)

print(p)

0.0001


In [151]:
rf_fig_path = f"{fig_path}/mix_EMG_ACC_figures/external_val_permutation_hist_RF.svg"

plot_acc_hist(sign_border_rf, PRED_ACC_rf, rand_accuracies_rf, rf_fig_path)

### plot mit y_true und allen y_preds ###

In [152]:
def normalize(s):
    return str(s).strip().lower()

def encode_str_labels(y_str, enc):
    y_norm = [normalize(v) for v in y_str]
    # Check auf unbekannte Labels
    unknown = sorted(set(y_norm) - set(enc.keys()))
    if unknown:
        raise ValueError(f"Unknown labels: {unknown} (expected: {list(enc.keys())})")
    return np.array([enc[v] for v in y_norm], dtype=float)

def with_offset(y, off, clamp=True):
    y_off = y.astype(float) + off
    if clamp:
        y_off = np.clip(y_off, -0.49, 2.49)
    return y_off

In [202]:
y_preds_fig_path = os.path.join(fig_path,"mix_EMG_ACC_figures", "y_true_LR_preds.svg")

# correct order
order = ['rest', 'move', 'suppr']
enc = {lbl: i for i, lbl in enumerate(order)}

# y_test_true / y_pred_lr: classes as strings here already!
y_true_num = encode_str_labels(y_test_true, enc) # encode back to nums
y_pred_num_lr = encode_str_labels(y_pred_lr, enc)
y_pred_num_knn = encode_str_labels(y_pred_knn, enc)
y_pred_num_rf = encode_str_labels(y_pred_rf, enc)

times = np.arange(len(y_true_num))

#offsets = {
#   "LR":  +0.03,
#   "KNN": -0.03,
#   "RF":  +0.00,
#}

fig, ax = plt.subplots(1,1, figsize=(12,6), constrained_layout=True)

ax.step(times, y_true_num, where='post', label='y_true',
        linewidth=7, color='lightgray')

#ax.step(times, with_offset(y_pred_num_lr,  offsets["LR"]),  where='post',
#        label='y_pred_LR',  linewidth=2.0, alpha=0.8, color='tab:blue')
#ax.step(times, with_offset(y_pred_num_knn, offsets["KNN"]), where='post',
#        label='y_pred_KNN', linewidth=2.0, alpha=0.8, color='tab:green')
#ax.step(times, with_offset(y_pred_num_rf,  offsets["RF"]),  where='post',
#        label='y_pred_RF',  linewidth=2.0, alpha=0.8, color='tab:orange')

ax.step(times, y_pred_num_lr, where='post',
        label='y_pred_LR',  linewidth=1.75, alpha=0.9, color='tab:blue')


ax.set_ylim(-0.25, 2.25)
ax.set_yticks([0,1,2])
ax.set_yticklabels(order)  # 0=rest on the bottom, 1=move, 2=suppression on top
ax.set_xlabel('samples')
ax.set_facecolor("white")
ax.legend(bbox_to_anchor=(1.02,0.5), loc="center left", frameon=False)

sp = ax.spines["bottom"]
sp.set_visible(True)
sp.set_linewidth(0.8)
sp.set_color("black")
sp.set_zorder(10)  

ax.tick_params(bottom=True, left=True, )  
plt.tight_layout()
#plt.show()
plt.savefig(y_preds_fig_path, format="svg", dpi=300, bbox_inches="tight", pad_inches=0.05)

C:\Users\User\AppData\Local\Temp\ipykernel_8556\2740463094.py:51: UserWarning: The figure layout has changed to tight
  plt.tight_layout()


### Sensitivitäten und Spezifitäten rauskriegen ###

In [109]:
importlib.reload(move_functions)

<module 'combined_analysis_bachelor.code.movement_detection_functions' from 'C:\\Users\\User\\Documents\\GitHub\\lid_opm\\combined_analysis_bachelor\\code\\movement_detection_functions.py'>

In [110]:
LR_dict = move_functions.per_class_sens_spec(y_test_true, y_pred_lr, class_names=["rest", "move", "suppr"])

Accuracy: 0.523
rest: sensitivity = 0.268, specificity = 0.991
move: sensitivity = 0.862, specificity = 0.968
suppr: sensitivity = 0.980, specificity = 0.451


In [111]:
KNN_dict = move_functions.per_class_sens_spec(y_test_true, y_pred_knn, class_names=["rest", "move", "suppr"])

Accuracy: 0.538
rest: sensitivity = 0.256, specificity = 1.000
move: sensitivity = 0.983, specificity = 0.840
suppr: sensitivity = 0.961, specificity = 0.588


In [112]:
RF_dict = move_functions.per_class_sens_spec(y_test_true, y_pred_rf, class_names=["rest", "move", "suppr"])

Accuracy: 0.531
rest: sensitivity = 0.250, specificity = 1.000
move: sensitivity = 0.966, specificity = 0.886
suppr: sensitivity = 0.961, specificity = 0.535
